<a href="https://colab.research.google.com/github/aamanp/DIMEMEX-IberLEF-2024/blob/main/DIMEMEX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#analizar texto, generar etiquetas. Analizar las imagenes, se generan etiquetas. Y por último se hace un voting.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
train= pd.read_csv('drive/MyDrive/train_data.csv', names=["imagen", "texto"], index_col=0) #con el index_col=0, le decimos que introduzca un indice en la columna 0

In [ ]:
train.head(20)

,texto
imagen,
DS_IMG_1075,Cuando te das cuenta que no es mugre @alexesmu
DS_IMG_2669,Ya estaríamos en octavos; si alguien no fallab...
DS_IMG_2593,Nadie: Lo de Cinepolis que se quedaron con tod...
DS_IMG_261,Dice que los tragos hicieron estrago en su cab...
DS_IMG_1037,Cuando te salen tus 3 novias al mismo tiempo O...
DS_IMG_2318,al menos mi cuñada me quiere mi cuñada
DS_IMG_3069,Quand [egas a@ plnte] Conalep conalep iniun I ...
DS_IMG_1879,YA NO PUBLICAREMEMES OFENSIVOS Y DE MAL GUSTO ...
DS_IMG_2435,SE TRANSFORMA EN VAMPIRO NO LO DEJAN SALIR DE ...


In [ ]:
train_labels= pd.read_csv('drive/MyDrive/train-labels_subtask_1.csv', names=["0", "1", "2"])
train_labels.head()
train_labels=train_labels.idxmax(axis=1)
train["labels"] = train_labels.to_list()

In [ ]:
#train.head()
train['labels'].value_counts()

labels
2    1405
1     472
0     386
Name: count, dtype: int64

In [ ]:
# Eliminar espacios
def eliminar_espacios(text):
    return  " ".join(text.split())

# To lower
def texto_to_lower(text):
  return text.lower()

from tqdm.autonotebook import tqdm
tqdm.pandas()

train["normalized"] = train["texto"].progress_apply(lambda x: eliminar_espacios(x))
train["normalized"] = train["normalized"].progress_apply(lambda x: texto_to_lower(x))

train.head()

<ipython-input-8-435f97408553>:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/2263 [00:00<?, ?it/s]

  0%|          | 0/2263 [00:00<?, ?it/s]

,texto,labels,normalized
imagen,,,
DS_IMG_1075,Cuando te das cuenta que no es mugre @alexesmu,0,cuando te das cuenta que no es mugre @alexesmu
DS_IMG_2669,Ya estaríamos en octavos; si alguien no fallab...,2,ya estaríamos en octavos; si alguien no fallab...
DS_IMG_2593,Nadie: Lo de Cinepolis que se quedaron con tod...,2,nadie: lo de cinepolis que se quedaron con tod...
DS_IMG_261,Dice que los tragos hicieron estrago en su cab...,0,dice que los tragos hicieron estrago en su cab...
DS_IMG_1037,Cuando te salen tus 3 novias al mismo tiempo O...,2,cuando te salen tus 3 novias al mismo tiempo o...


In [ ]:
from nltk.tokenize import TweetTokenizer
texto_tokenizer = TweetTokenizer()

def normalizar_tokens(documento):
    # Dividir el documento en palabras
    palabras =  texto_tokenizer.tokenize(documento) # tokenize(documento)

    # Reemplazar URLs, menciones de Twitter y números por los tokens correspondientes
    for i in range(len(palabras)):
        if palabras[i].startswith("http://") or palabras[i].startswith("https://") or palabras[i].startswith("www."):
            palabras[i] = "URL"
        elif palabras[i].startswith("@"): #startswith "@"
            palabras[i] = "MENTION"
        elif palabras[i].isdigit(): # .isdigit()
            palabras[i] = "NUM"

    # Unir las palabras de nuevo en un documento modificado
    documento_modificado = ' '.join(palabras)

    return documento_modificado

In [ ]:
train["normalized"] = train["normalized"].progress_apply(lambda x: normalizar_tokens(x))
train.head()

  0%|          | 0/2263 [00:00<?, ?it/s]

,texto,labels,normalized
imagen,,,
DS_IMG_1075,Cuando te das cuenta que no es mugre @alexesmu,0,cuando te das cuenta que no es mugre MENTION
DS_IMG_2669,Ya estaríamos en octavos; si alguien no fallab...,2,ya estaríamos en octavos ; si alguien no falla...
DS_IMG_2593,Nadie: Lo de Cinepolis que se quedaron con tod...,2,nadie : lo de cinepolis que se quedaron con to...
DS_IMG_261,Dice que los tragos hicieron estrago en su cab...,0,dice que los tragos hicieron estrago en su cab...
DS_IMG_1037,Cuando te salen tus 3 novias al mismo tiempo O...,2,cuando te salen tus NUM novias al mismo tiempo...


In [ ]:
!python3 -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 33.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
nlp = spacy.load('es_core_news_sm', disable=['parser', 'ner'])


import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
en_stopwords = stopwords.words('spanish')

def lematizar_eliminacion_tokens(texto):
    # Procesar el texto con el objeto nlp
    doc = nlp(texto)

    # Lematizar el texto
    lemas = [token.lemma_ for token in doc]

    # Eliminar símbolos de puntuación y stopwords
    tokens_filtrados = [token for token in lemas if token.isalpha() and token.lower() not in en_stopwords]

    # Unir los tokens filtrados en un nuevo texto
    texto_procesado = ' '.join(tokens_filtrados)

    return texto_procesado

train["normalized"] = train["normalized"].progress_apply(lambda x: lematizar_eliminacion_tokens(x))
train.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


  0%|          | 0/2263 [00:00<?, ?it/s]

,texto,labels,normalized
imagen,,,
DS_IMG_1075,Cuando te das cuenta que no es mugre @alexesmu,0,dar cuenta ser mugre MENTION
DS_IMG_2669,Ya estaríamos en octavos; si alguien no fallab...,2,estaríar octavo si alguien fallar NUM mexico z...
DS_IMG_2593,Nadie: Lo de Cinepolis que se quedaron con tod...,2,nadie cinepoli quedar vaso barbie NUM NUM NUM ...
DS_IMG_261,Dice que los tragos hicieron estrago en su cab...,0,decir trago hacer estrago cabeza entonces ser ...
DS_IMG_1037,Cuando te salen tus 3 novias al mismo tiempo O...,2,salir NUM novia mismo tiempo ok cagar mejor ir...


In [ ]:
def token_appearance(document, token = "MENTION"):
  return len([x for x in document.split() if x == token])

train["num_mention"] = train["normalized"].progress_apply(lambda x: token_appearance(x, token="MENTION"))
train["num_URLS"] = train["normalized"].progress_apply(lambda x: token_appearance(x, token="URL"))
train["num_NUM"] = train["normalized"].progress_apply(lambda x: token_appearance(x, token="NUM"))

  0%|          | 0/2263 [00:00<?, ?it/s]

  0%|          | 0/2263 [00:00<?, ?it/s]

  0%|          | 0/2263 [00:00<?, ?it/s]

In [ ]:
train

,texto,labels,normalized,num_mention,num_URLS,num_NUM
imagen,,,,,,
DS_IMG_1075,Cuando te das cuenta que no es mugre @alexesmu,0,dar cuenta ser mugre MENTION,1,0,0
DS_IMG_2669,Ya estaríamos en octavos; si alguien no fallab...,2,estaríar octavo si alguien fallar NUM mexico z...,0,0,1
DS_IMG_2593,Nadie: Lo de Cinepolis que se quedaron con tod...,2,nadie cinepoli quedar vaso barbie NUM NUM NUM ...,0,0,6
DS_IMG_261,Dice que los tragos hicieron estrago en su cab...,0,decir trago hacer estrago cabeza entonces ser ...,0,0,0
DS_IMG_1037,Cuando te salen tus 3 novias al mismo tiempo O...,2,salir NUM novia mismo tiempo ok cagar mejor ir...,0,0,1
...,...,...,...,...,...,...
DS_IMG_456,Elhomofóbico promedio,1,elhomofóbico promedio,0,0,0
DS_IMG_642,Cuando te quieres pajear pero tu perro no se v...,1,querer pajear perro ir mematic ne MENTION estr...,1,0,0
DS_IMG_2480,CUANDO YA ESTAS LISTO PARA DAR EL GRITO CON TU...,2,listo dar grito bolillo cualquiera cosa favor ...,0,0,0


In [ ]:
train_imagenes = pd.read_csv("drive/MyDrive/DIMEMEX/csv_imagenes/train.csv", index_col=0)
train_imagenes

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
DS_IMG_508,0.558996,-0.100422,-0.153666,0.358985,0.450803,-0.308662,0.344734,-0.489874,-0.619295,-0.565982,...,0.065817,-0.349862,0.071270,0.370170,0.077820,-0.154476,-0.165892,-0.164343,-0.372248,0.255372
DS_IMG_584,0.212567,-0.117731,-0.307430,0.469084,0.019655,-0.356494,0.363694,0.501771,0.005585,-0.374570,...,0.296676,-0.228208,0.466057,0.068594,0.174292,-0.329361,0.436264,0.351559,0.102795,0.339749
DS_IMG_2523,0.606540,0.768875,-0.713259,-0.522802,-0.315809,0.175691,-0.224950,0.366440,0.368168,-0.580241,...,0.747420,0.676396,0.658944,-0.521409,0.358419,-0.179088,0.011135,0.061832,-0.498169,0.776241
DS_IMG_589,0.069092,0.290318,-0.696644,-0.795009,0.177350,0.250823,-0.581107,0.030569,0.158775,-0.701717,...,-0.214464,-0.250606,0.786350,-0.498171,-0.160765,-0.326590,-0.416686,0.331526,0.092866,0.304960
DS_IMG_1930,-0.155782,-0.388472,0.401735,0.196331,0.316715,0.602845,0.492766,0.523379,-0.091211,-0.243360,...,-0.353552,-0.480468,0.598224,0.358441,-0.267116,0.440057,0.550230,-0.568605,0.746728,0.153550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DS_IMG_2484,0.247957,0.610536,-0.214570,0.207667,-0.391415,-0.476930,0.621023,0.283322,-0.587913,-0.456450,...,-0.328447,-0.829180,0.214176,-0.646141,0.502820,0.254461,-0.052018,0.109844,0.173194,0.247560
DS_IMG_2317,0.756452,-0.152649,-0.339555,0.327620,-0.007929,0.222874,-0.335313,-0.162786,0.004944,-0.198641,...,0.194817,0.727798,0.601153,-0.611670,-0.150360,-0.135819,0.481967,-0.541784,-0.781074,0.306147
DS_IMG_2002,-0.138957,0.472676,-0.859473,0.059980,0.706658,0.844839,0.467559,-0.393158,-0.106201,-0.260379,...,-0.364342,-0.544982,0.616728,0.180150,-0.522292,-0.274436,-0.108123,-0.417499,0.372097,-0.597889
DS_IMG_1428,-0.023445,-0.180461,-0.272871,0.097721,0.080339,0.076493,-0.211595,0.426029,0.148754,0.264527,...,-0.247115,-0.467443,0.423945,-0.178072,-0.397214,-0.903454,-0.184345,-0.055817,0.122331,0.257193


In [ ]:
train_combinado = train.merge(train_imagenes, left_index=True, right_index=True)
train_combinado

,texto,labels,normalized,num_mention,num_URLS,num_NUM,0,1,2,3,...,758,759,760,761,762,763,764,765,766,767
DS_IMG_1075,Cuando te das cuenta que no es mugre @alexesmu,0,dar cuenta ser mugre MENTION,1,0,0,0.645640,-0.514144,0.264805,-0.022821,...,0.495025,0.143103,0.325312,-0.779648,-0.063925,-0.093646,-0.108067,0.049688,-0.261500,0.100154
DS_IMG_2669,Ya estaríamos en octavos; si alguien no fallab...,2,estaríar octavo si alguien fallar NUM mexico z...,0,0,1,0.334528,0.513736,0.140332,0.851287,...,0.035021,-0.170740,0.705473,-0.758880,0.178379,-0.121114,0.632025,0.181586,-0.409698,-0.039597
DS_IMG_2593,Nadie: Lo de Cinepolis que se quedaron con tod...,2,nadie cinepoli quedar vaso barbie NUM NUM NUM ...,0,0,6,0.364959,-0.426154,0.356037,-0.224330,...,0.002726,0.084866,-0.195790,-0.419828,0.605474,-0.733779,-0.576938,0.507961,-0.447820,0.475276
DS_IMG_261,Dice que los tragos hicieron estrago en su cab...,0,decir trago hacer estrago cabeza entonces ser ...,0,0,0,0.506989,0.375483,0.166320,-0.231450,...,0.031333,0.540699,0.755823,-0.517846,0.042687,-0.001421,0.809948,-0.432517,0.037742,-0.110795
DS_IMG_1037,Cuando te salen tus 3 novias al mismo tiempo O...,2,salir NUM novia mismo tiempo ok cagar mejor ir...,0,0,1,0.222385,-0.597465,0.088138,-0.049229,...,-0.405820,-0.152219,0.503510,-0.662006,-0.075695,-0.780589,0.091075,-0.716168,-0.042253,-0.144587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DS_IMG_456,Elhomofóbico promedio,1,elhomofóbico promedio,0,0,0,0.031500,0.023711,0.101960,0.009865,...,-0.897147,-0.572755,0.404213,0.184019,0.302295,0.302241,0.344189,0.059415,-0.097819,0.556787
DS_IMG_642,Cuando te quieres pajear pero tu perro no se v...,1,querer pajear perro ir mematic ne MENTION estr...,1,0,0,-0.199391,-0.736236,0.391898,0.307867,...,-0.255539,-0.600474,-0.076181,-0.535937,0.025196,-0.300825,0.020409,0.176902,-0.083852,0.519161
DS_IMG_2480,CUANDO YA ESTAS LISTO PARA DAR EL GRITO CON TU...,2,listo dar grito bolillo cualquiera cosa favor ...,0,0,0,0.012206,-0.161433,-0.045054,0.195448,...,-0.299800,-0.696423,0.163360,-0.576277,0.124446,-0.133454,-0.219410,-0.254735,-0.524906,-0.459644
DS_IMG_327,Cuando Ix acabas de conocer y te dice No soy f...,2,ix acaba conocer decir ser feminista machista ...,0,0,0,0.212429,-0.138232,-0.596973,-0.503587,...,0.014685,0.529638,0.614074,0.230072,0.454410,-0.572181,0.492079,0.207271,-0.370096,0.144851


In [ ]:
train_combinado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2263 entries, DS_IMG_1075 to DS_IMG_1190
Columns: 774 entries, texto to 767
dtypes: float64(768), int64(3), object(3)
memory usage: 13.4+ MB


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizador = TfidfVectorizer(lowercase=False,
                               ngram_range=(1,4),
                               min_df=0.001)

train_text_embedding = vectorizador.fit_transform(train["normalized"]).toarray()


Ahora lo junto con el train. Para hacerlo fácil, como ya he extraido el embedding, elimino la columna de texto.

In [ ]:
y = train_combinado["labels"]
X = np.concatenate((train_text_embedding, train_combinado.drop(columns=["texto", "normalized", "labels"]).values), axis=1)

In [ ]:
X.shape

(2263, 1953)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=0,
                                                    stratify = y)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

In [ ]:
model = MLPClassifier()
solver= ['lbfgs', 'sgd', 'adam']
alpha= [1e-5]
#hidden_layer_sizes= [[100, 50],[100,75], [25,5]]
hidden_layer_sizes= [100,75,5]
random_state= [1]
learning_rate_init= [0.05]
batch_size= [500,200]

grid = dict(solver=solver,alpha=alpha,hidden_layer_sizes=hidden_layer_sizes,random_state=random_state, learning_rate_init=learning_rate_init, batch_size=batch_size)
cv = StratifiedKFold(n_splits=5)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='f1_macro',error_score=0)

grid_result = grid_search.fit(X_train, y_train)

print("Mejor f1: %f usando los parámetros %s" % (grid_result.best_score_, grid_result.best_params_))

Mejor f1: 0.398772 usando los parámetros {'alpha': 1e-05, 'batch_size': 500, 'hidden_layer_sizes': 100, 'learning_rate_init': 0.05, 'random_state': 1, 'solver': 'lbfgs'}


In [ ]:
# define models and parameters
model = LogisticRegression()

#solvers = ['newton-cg', 'liblinear']
#penalty = ['l2', 'l1']
#c_values = [100, 10, 1.0, 0.1, 0.01]
#class_weight=["balanced", None]


solvers = ['liblinear']
penalty = ['l2']
c_values = [1.0, 0.1]
class_weight=["balanced"]

grid = dict(solver=solvers,penalty=penalty,C=c_values, class_weight=class_weight)
cv = StratifiedKFold(n_splits=5)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='f1_macro',error_score=0)

grid_result = grid_search.fit(X_train, y_train)
# summarize results
print("Mejor f1: %f usando los parámetros %s" % (grid_result.best_score_, grid_result.best_params_))


Mejor f1: 0.420697 usando los parámetros {'C': 0.1, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'liblinear'}


In [ ]:
model = RandomForestClassifier()
n_estimators = [50,100,150]
max_depth = [23]
min_samples_leaf = [5,10]
max_features = [5,10,15]
class_weight=["balanced", None]

grid = dict(n_estimators=n_estimators,max_depth=max_depth,min_samples_leaf=min_samples_leaf, max_features = max_features, class_weight=class_weight)
cv = StratifiedKFold(n_splits=5)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='f1_macro',error_score=0)

grid_result = grid_search.fit(X_train, y_train)
# summarize results
print("Mejor f1: %f usando los parámetros %s" % (grid_result.best_score_, grid_result.best_params_))

Mejor f1: 0.363894 usando los parámetros {'class_weight': 'balanced', 'max_depth': 23, 'max_features': 5, 'min_samples_leaf': 10, 'n_estimators': 100}


In [ ]:
model = LogisticRegression(C=1, penalty= 'l2', solver= 'newton-cg', class_weight="balanced")
model.fit(X_train,y_train)
y_predict= model.predict(X_test)


In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,y_predict))
print(classification_report(y_test,y_predict))

[[ 35  16  46]
 [ 26  31  61]
 [ 64  82 205]]
              precision    recall  f1-score   support

           0       0.28      0.36      0.32        97
           1       0.24      0.26      0.25       118
           2       0.66      0.58      0.62       351

    accuracy                           0.48       566
   macro avg       0.39      0.40      0.39       566
weighted avg       0.51      0.48      0.49       566



Validation y Test.



In [ ]:
# Entrenamos con todos los datos de entrenamiento
#model = LogisticRegression(C=1, penalty= 'l2', solver= 'newton-cg', class_weight="balanced")
model = MLPClassifier(alpha= 1e-05, batch_size= 500, hidden_layer_sizes= 5, learning_rate_init= 0.05, random_state= 1, solver='sgd')
model.fit(X,y)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(alpha=1e-05, batch_size=500, hidden_layer_sizes=5,
              learning_rate_init=0.05, random_state=1, solver='sgd')

In [ ]:
def convertir_csv_a_X(csv_texto, csv_imagen, vectorizador):
  df= pd.read_csv(csv_texto, names=["imagen", "texto"], index_col=0)

  df["normalized"] = df["texto"].progress_apply(lambda x: eliminar_espacios(x))
  df["normalized"] = df["normalized"].progress_apply(lambda x: texto_to_lower(x))
  df["normalized"] = df["normalized"].progress_apply(lambda x: normalizar_tokens(x))
  df["normalized"] = df["normalized"].progress_apply(lambda x: lematizar_eliminacion_tokens(x))
  df["num_mention"] = df["normalized"].progress_apply(lambda x: token_appearance(x, token="MENTION"))
  df["num_URLS"] = df["normalized"].progress_apply(lambda x: token_appearance(x, token="URL"))
  df["num_NUM"] = df["normalized"].progress_apply(lambda x: token_appearance(x, token="NUM"))

  df_imagenes = pd.read_csv(csv_imagen, index_col=0)
  df_combinado = df.merge(df_imagenes, left_index=True, right_index=True)

  df_text_embedding = vectorizador.transform(df["normalized"]).toarray()
  df_combinado = df_combinado.drop(columns=["texto", "normalized"])
  X = np.concatenate((df_text_embedding, df_combinado.values), axis=1)
  return X

In [ ]:
X_validation = convertir_csv_a_X(csv_texto="drive/MyDrive/validation_data.csv", csv_imagen="drive/MyDrive/DIMEMEX/csv_imagenes/validation.csv", vectorizador=vectorizador)

  0%|          | 0/323 [00:00<?, ?it/s]

  0%|          | 0/323 [00:00<?, ?it/s]

  0%|          | 0/323 [00:00<?, ?it/s]

  0%|          | 0/323 [00:00<?, ?it/s]

  0%|          | 0/323 [00:00<?, ?it/s]

  0%|          | 0/323 [00:00<?, ?it/s]

  0%|          | 0/323 [00:00<?, ?it/s]

In [ ]:
y_predict = model.predict(X_validation)
y_predict

array(['2', '2', '2', '2', '1', '1', '2', '2', '2', '2', '1', '2', '2',
       '0', '0', '1', '1', '0', '2', '2', '0', '2', '2', '1', '0', '0',
       '1', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2',
       '0', '2', '2', '2', '1', '2', '1', '2', '2', '2', '2', '2', '0',
       '1', '2', '0', '0', '1', '2', '2', '0', '1', '2', '2', '2', '2',
       '1', '2', '2', '2', '1', '2', '2', '0', '2', '2', '1', '2', '2',
       '2', '2', '2', '2', '2', '2', '2', '2', '0', '1', '1', '2', '2',
       '2', '2', '0', '2', '1', '2', '2', '1', '2', '1', '0', '2', '0',
       '1', '2', '2', '1', '1', '2', '2', '1', '0', '2', '1', '2', '2',
       '2', '2', '2', '2', '2', '2', '0', '2', '2', '1', '2', '1', '2',
       '2', '1', '0', '2', '2', '1', '2', '2', '1', '2', '2', '1', '1',
       '1', '2', '1', '2', '1', '2', '2', '0', '1', '2', '2', '1', '1',
       '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '1', '1', '2',
       '2', '2', '2', '0', '2', '2', '2', '0', '2', '2', '2', '0

In [ ]:
df = pd.DataFrame(y_predict)
df_dummies = pd.get_dummies(df, dtype=int)
df_dummies.to_csv("drive/MyDrive/submission1.csv", header=False, index=False)

In [ ]:
X_test = convertir_csv_a_X(csv_texto="drive/MyDrive/test_data.csv", csv_imagen="drive/MyDrive/DIMEMEX/csv_imagenes/test.csv", vectorizador=vectorizador)

  0%|          | 0/649 [00:00<?, ?it/s]

  0%|          | 0/649 [00:00<?, ?it/s]

  0%|          | 0/649 [00:00<?, ?it/s]

  0%|          | 0/649 [00:00<?, ?it/s]

  0%|          | 0/649 [00:00<?, ?it/s]

  0%|          | 0/649 [00:00<?, ?it/s]

  0%|          | 0/649 [00:00<?, ?it/s]

In [ ]:
y_predict = model.predict(X_test)
y_predict

array(['1', '1', '2', '1', '0', '2', '1', '1', '1', '2', '2', '2', '1',
       '1', '2', '2', '1', '1', '1', '1', '2', '2', '1', '2', '1', '2',
       '0', '1', '2', '1', '2', '2', '1', '1', '2', '1', '1', '1', '1',
       '1', '2', '2', '1', '1', '1', '2', '2', '2', '1', '1', '1', '1',
       '1', '1', '2', '1', '1', '2', '2', '1', '1', '2', '1', '2', '0',
       '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '0', '1', '1', '1', '0', '1', '1', '2', '1', '1',
       '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '2', '1',
       '2', '1', '0', '1', '1', '1', '2', '2', '2', '1', '1', '1', '0',
       '2', '1', '1', '1', '1', '2', '1', '1', '2', '1', '2', '1', '1',
       '1', '2', '1', '2', '1', '2', '2', '1', '1', '2', '2', '1', '2',
       '1', '0', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1',
       '1', '2', '2', '1', '2', '1', '0', '0', '2', '2', '1', '1', '2',
       '2', '2', '1', '1', '1', '1', '0', '2', '2', '2', '1', '1

In [ ]:
df = pd.DataFrame(y_predict)
df_dummies = pd.get_dummies(df, dtype=int)
df_dummies.to_csv("drive/MyDrive/testsubmission1.csv", header=False, index=False)